## Install important libraries

In [6]:
pip install requests pandas pyodbc sqlalchemy


Note: you may need to restart the kernel to use updated packages.


# Collect Weather Data Using a Public API

In [8]:
import requests
from datetime import datetime

# API Configuration
API_KEY = "0eb62c4c6c41a6aba80ac4292fd0213a" 
CITIES = [
    "Cairo", "London", "New York", "Tokyo", "Paris", "Berlin", "Moscow", "Dubai", 
    "Sydney", "Toronto", "Beijing", "Mumbai", "Rome", "Madrid", "Seoul", "Singapore", 
    "Los Angeles", "Chicago", "São Paulo", "Bangkok", "Istanbul", "Jakarta", "Lagos", 
    "Cape Town", "Buenos Aires", "Mexico City", "Mumbai", "Shanghai", "Moscow", "Delhi"]

BASE_URL = "http://api.openweathermap.org/data/2.5/weather"

# Function to fetch weather data for a city
def fetch_weather_data(city):
    try:
        url = f"{BASE_URL}?q={city}&appid={API_KEY}&units=metric"
        
        # Make the API request
        response = requests.get(url)
        response.raise_for_status()  
        weather_data = response.json()

        # Check if the required keys exist in the response
        if all(key in weather_data for key in ['name', 'main', 'weather']):
            data = {
                "city": weather_data["name"],
                "temperature": weather_data["main"]["temp"],
                "humidity": weather_data["main"]["humidity"],
                "weather": weather_data["weather"][0]["description"],
                "timestamp": datetime.utcfromtimestamp(weather_data["dt"]).strftime('%Y-%m-%d %H:%M:%S')  # Convert timestamp
            }
            return data
        else:
            print(f"Error: Unexpected API response format for {city}. Please check the API key and city name.")
            return None

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {city}: {e}")
        return None

# Fetch weather data for all cities
for city in CITIES:
    weather_data = fetch_weather_data(city)
    if weather_data:
        print(f"Weather Data for {city}:")
        print(weather_data)
    else:
        print(f"No data available for {city}.")
    print("-" * 40)  # Separator for readability

Weather Data for Cairo:
{'city': 'Cairo', 'temperature': 20.42, 'humidity': 52, 'weather': 'broken clouds', 'timestamp': '2025-02-05 10:57:22'}
----------------------------------------
Weather Data for London:
{'city': 'London', 'temperature': 7.55, 'humidity': 83, 'weather': 'broken clouds', 'timestamp': '2025-02-05 11:00:22'}
----------------------------------------
Weather Data for New York:
{'city': 'New York', 'temperature': -2.49, 'humidity': 52, 'weather': 'overcast clouds', 'timestamp': '2025-02-05 11:00:19'}
----------------------------------------
Weather Data for Tokyo:
{'city': 'Tokyo', 'temperature': 3.74, 'humidity': 31, 'weather': 'few clouds', 'timestamp': '2025-02-05 11:05:10'}
----------------------------------------
Weather Data for Paris:
{'city': 'Paris', 'temperature': 5.16, 'humidity': 100, 'weather': 'fog', 'timestamp': '2025-02-05 10:54:53'}
----------------------------------------
Weather Data for Berlin:
{'city': 'Berlin', 'temperature': 2.03, 'humidity': 86,

# Connection to local database, and send data from Weather API

In [9]:
import pyodbc

# Connect to Azure SQL
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=ZAHRAMO7Y\MYSERVER;' 
    'DATABASE=weather_db;'
    'Trusted_Connection=yes;'
)

cursor = conn.cursor()

# Insert weather data for all cities
for city in CITIES:
    weather_data = fetch_weather_data(city)
    if weather_data:
        print(f"Weather Data for {city}:")
        print(weather_data)
        # Insert weather data
        cursor.execute("""
            INSERT INTO weather (city, temperature, humidity, weather, timestamp)
            VALUES (?, ?, ?, ?, ?)""",
            (weather_data["city"], weather_data["temperature"], weather_data["humidity"], weather_data["weather"], weather_data["timestamp"])
        )
        print("Data inserted successfully!")
    else:
        print(f"No data available for {city}.")
    print("-" * 40)  # Separator for readability

conn.commit()
cursor.close()
conn.close()


Weather Data for Cairo:
{'city': 'Cairo', 'temperature': 20.42, 'humidity': 52, 'weather': 'broken clouds', 'timestamp': '2025-02-05 10:57:22'}
Data inserted successfully!
----------------------------------------
Weather Data for London:
{'city': 'London', 'temperature': 7.55, 'humidity': 83, 'weather': 'broken clouds', 'timestamp': '2025-02-05 11:00:22'}
Data inserted successfully!
----------------------------------------
Weather Data for New York:
{'city': 'New York', 'temperature': -2.49, 'humidity': 52, 'weather': 'overcast clouds', 'timestamp': '2025-02-05 11:00:19'}
Data inserted successfully!
----------------------------------------
Weather Data for Tokyo:
{'city': 'Tokyo', 'temperature': 3.74, 'humidity': 31, 'weather': 'few clouds', 'timestamp': '2025-02-05 11:05:10'}
Data inserted successfully!
----------------------------------------
Weather Data for Paris:
{'city': 'Paris', 'temperature': 5.42, 'humidity': 99, 'weather': 'fog', 'timestamp': '2025-02-05 11:07:25'}
Data inser

# Transorm and clean data

In [10]:
import pandas as pd
from sqlalchemy import create_engine

# Create a SQLAlchemy engine
connection_string = (
    "mssql+pyodbc://ZAHRAMO7Y\MYSERVER/weather_db?"
    "driver=ODBC+Driver+17+for+SQL+Server&"
    "Trusted_Connection=yes"
)
engine = create_engine(connection_string)

# Query the database
query = "SELECT * FROM weather;"
df = pd.read_sql(query, engine)

# Convert timestamp
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Convert temperature to Fahrenheit
df["temperature_f"] = df["temperature"] * 9/5 + 32

# Drop duplicate rows, ignoring the timestamp column
df = df.drop_duplicates(subset=["city", "temperature", "humidity", "weather"])

# Display the first 30 rows
print(df.head(30))


    id          city  temperature  humidity           weather  \
0    1         Cairo        16.42        67  scattered clouds   
5    6        London         8.58        86   overcast clouds   
6    7      New York         4.93        38        few clouds   
7    8         Tokyo         1.51        55        few clouds   
8    9         Paris         2.12        99               fog   
9   10        Berlin        -1.24        89         clear sky   
10  11        Moscow        -0.74        67   overcast clouds   
11  12         Dubai        18.96        55         clear sky   
12  13        Sydney        24.34        87         clear sky   
13  14       Toronto        -5.76        68     broken clouds   
14  15       Beijing        -8.06        24         clear sky   
15  16        Mumbai        23.99        33             smoke   
16  17          Rome        24.54        50         clear sky   
17  18        Madrid         6.43        69         clear sky   
18  19         Seoul     